In [13]:
import plaidml.keras
plaidml.keras.install_backend()

from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator

import cv2

import pandas as pd
import numpy as np
import time
import itertools

In [14]:
# GLOBAL VARS
TRAIN_DIR = 'asl-alphabet/asl_alphabet_train'
TEST_DIR = 'asl-alphabet/asl_alphabet_test'

TARGET_SIZE = (64, 64)
TARGET_DIMS = (64, 64, 3) # add channel for RGB
N_CLASSES = 29
VALIDATION_SPLIT = 0.15
BATCH_SIZE = 64

# Model saving for easier local iterations
MODEL_DIR = 'model3'
MODEL_PATH = MODEL_DIR + '/cnn-model.h5'
MODEL_WEIGHTS_PATH = MODEL_DIR + '/cnn-model.weights.h5'
SAVE = True

In [18]:
def make_generator(group):
    
    # used in preprocessing the image
    augmentor_options = {
        'samplewise_center': True,
        'samplewise_std_normalization': True,
        'preprocessing_function':lambda x: cv2.Sobel(x, cv2.CV_64F, 0, 1, ksize=5),
        'validation_split': VALIDATION_SPLIT
    }
    
    # used in reading from directory
    flow_options = {
        'target_size': TARGET_SIZE,
        'batch_size': BATCH_SIZE,
        'shuffle': True,
        'subset': group
    }

    generator = ImageDataGenerator(**augmentor_options)
    return generator.flow_from_directory(TRAIN_DIR, **flow_options)

In [20]:
def build_model(save=False):
    
    model = Sequential()
    model.add(Conv2D(64, kernel_size=4, strides=1, activation='relu', input_shape=TARGET_DIMS))
#     model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu'))
    model.add(Dropout(0.4))
#     model.add(MaxPooling2D())
#     model.add(Conv2D(128, kernel_size=4, strides=1, activation='relu'))
#     model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Conv2D(256, kernel_size=4, strides=1, activation='relu'))
#     model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu'))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(N_CLASSES, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.save(MODEL_PATH) 
    return model

CNN_MODEL = build_model(save=SAVE)
print_summary(CNN_MODEL)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 61, 61, 64)        3136      
_________________________________________________________________
dropout_5 (Dropout)          (None, 61, 61, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 238144)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 238144)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               60965120  
_________________________________________________________________
dense_6 (Dense)              (None, 29)                7453      
Total params: 60,975,709
Trainable params: 60,975,709
Non-trainable params: 0
________________________________________________________________

In [21]:
def fit_model(model, train_generator, val_generator, save):
    fitted = model.fit_generator(train_generator, epochs=2, validation_data=val_generator)
    if save: fitted.save_weights(MODEL_WEIGHTS_PATH)
    return fitted


CNN_TRAIN_GENERATOR = make_generator('training')
CNN_VAL_GENERATOR = make_generator('validation')

start_time = time.time()
model = fit_model(CNN_MODEL, CNN_TRAIN_GENERATOR, CNN_VAL_GENERATOR, save=SAVE)
print('Fitting the model took ~{:.0f} second(s).'.format(time.time() - start_time))


Found 73950 images belonging to 29 classes.
Found 13050 images belonging to 29 classes.
Epoch 1/2
   2/1156 [..............................] - ETA: 46:21 - loss: 8.9949 - acc: 0.0859  

KeyboardInterrupt: 

In [11]:
print('Final Accuracy: {:.2f}%'.format(model.history['acc'][4] * 100))
print('Validation set accuracy: {:.2f}%'.format(model.history['val_acc'][4] * 100))

Final Accuracy: 94.81%
Validation set accuracy: 74.51%
